# Hugging Face "Tokenizers"와 PyTorch "Captum" 라이브러리 사용기

## 데이터 추출 및 훈련 데이터 생성

In [1]:
import pandas as pd

# NSMC 훈련용 코퍼스 내 문장 및 라벨 데이터 추출

f_train = pd.read_csv('data/ratings_train.txt', sep='\t')
train_pair = [(row[1], row[2]) for _, row in f_train.iterrows() if type(row[1]) == str]

train_data  = [pair[0] for pair in train_pair]
train_label = [pair[1] for pair in train_pair]

In [2]:
# 추출된 문장 및 라벨 데이터 일부 확인

for data, label in zip(train_data[:3], train_label[:3]):
    print(f'문장: {data}\n라벨: {label}\n')

문장: 아 더빙.. 진짜 짜증나네요 목소리
라벨: 0

문장: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
라벨: 1

문장: 너무재밓었다그래서보는것을추천한다
라벨: 0



## Tokenizers 라이브러리 활용한 토크나이저 학습

In [3]:
# 프로젝트 전반에 사용될 변수 사전 정의

params = {
    'batch_size': 64,
    'num_epoch': 10,
    'lr': 0.003,
    'dropout': 0.5,
    'min_frequency': 3,
    'max_len': 20,
    
    'vocab_size': 20000,
    'embed_dim': 100,
    'hidden_dim': 256,
    'filter_sizes': [2, 3, 4],
    'num_filters': 100,
    'output_dim': 1,
}

In [4]:
# 추출한 문장 데이터 텍스트 파일로 저장

with open('train_tokenizer.txt', 'w', encoding='utf-8') as f:
    for line in train_data:
        print(line, file=f)

In [5]:
from tokenizers import BPETokenizer

# BPE 토크나이저 초기화

tokenizer = BPETokenizer()


# 앞서 제작한 텍스트 파일 활용해 토크나이저 훈련

tokenizer.train(
    'train_tokenizer.txt',
    vocab_size=params['vocab_size'],
    min_frequency=params['min_frequency'],
    suffix='',
    special_tokens=['<PAD>', '<SOS>', '<EOS>', '<UNK>'],
)

In [6]:
# 패딩 토큰 인덱스 확인

print(tokenizer.token_to_id('<PAD>'))

0


In [7]:
# 스페셜 토큰 변수화

pad_idx = tokenizer.token_to_id('<PAD>')
sos_idx = tokenizer.token_to_id('<SOS>')
eos_idx = tokenizer.token_to_id('<EOS>')

In [8]:
# 토크나이저에 대해 패딩 옵션 설정

tokenizer.enable_padding(pad_id=pad_idx, pad_token='<PAD>', max_length=params['max_len'])

In [9]:
# 'encode_batch' 함수 이용해 훈련 데이터셋에 대해 토크나이즈 수행

encoded_data = tokenizer.encode_batch(train_data)

In [10]:
# 데이터 개수 확인

print(f'훈련 데이터:\t{len(train_data)} 개')
print(f'훈련 라벨:\t{len(train_label)} 개')
print(f'인코딩 데이터:\t{len(encoded_data)} 개')

훈련 데이터:	149995 개
훈련 라벨:	149995 개
인코딩 데이터:	149995 개


In [11]:
# 토크나이저 초기 훈련 결과 확인

print(f'토큰: {encoded_data[2020].tokens}\n')
print(f'아이디: {encoded_data[2020].ids}')

토큰: ['20', '여', '년이', '흘러', '두', '기억에', '남는', '사랑에', '대해', '생각해', '볼수있는', '유쾌한', '영화', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

아이디: [1216, 647, 1958, 2402, 291, 1665, 1310, 2877, 1827, 2162, 3922, 2863, 1005, 0, 0, 0, 0, 0, 0, 0]


## 토크나이즈 결과 후처리

In [12]:
# 단어 사전 구축

vocab = {}

for idx in range(20000):
    word = tokenizer.id_to_token(idx)
    id = tokenizer.token_to_id(word)
    vocab[word] = id

In [13]:
# 후처리 함수 정의

def postprocess(input_ids):
    input_ids = [sos_idx] + input_ids
    
    # 문장 최대 길이까지 슬라이싱
    input_ids = input_ids[:params['max_len']]

    # 패딩 토큰이 포함된 문장이라면 원 문장 말미에 <EOS> 토큰 삽입 
    if pad_idx in input_ids:
        pad_start = input_ids.index(pad_idx)
        input_ids[pad_start] = eos_idx

    # 패딩 토큰이 포함되지 않은 문장이라면, 시퀀스 말미에 <EOS> 토큰 삽입
    else:
        input_ids[-1] = eos_idx
    
    return input_ids

In [14]:
# 후처리 함수 이용해 토크나이즈 문장 후처리

processed_data = [postprocess(data.ids) for data in encoded_data]

In [15]:
# 후처리 결과 확인

print(f'후처리 결과: {processed_data[2020]}\n')
print(f'후처리 결과 디코딩: {tokenizer.decode(processed_data[2020])}')

후처리 결과: [1, 1216, 647, 1958, 2402, 291, 1665, 1310, 2877, 1827, 2162, 3922, 2863, 1005, 2, 0, 0, 0, 0, 0]

후처리 결과 디코딩: <SOS>20여년이흘러두기억에남는사랑에대해생각해볼수있는유쾌한영화<EOS><PAD><PAD><PAD><PAD><PAD>


# 간단한 CNN 문장 분류기 구현

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils import data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# CNN 모델 정의

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(params['vocab_size'], params['embed_dim'], padding_idx=pad_idx)

        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, 
                                              out_channels=params['num_filters'], 
                                              kernel_size=(fs, params['embed_dim'])) 
                                    for fs in params['filter_sizes']])
        
        self.fc = nn.Linear(len(params['filter_sizes']) * params['num_filters'], 1)
        
        self.dropout = nn.Dropout(params['dropout'])
        
    def forward(self, input_ids):
        # input_ids = [배치 사이즈, 문장 길이]

        embedded = self.embedding(input_ids).unsqueeze(1)
        # embedded = [배치 사이즈, 채널 개수, 임베딩 차원]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        # conved_n = [배치 사이즈, 필터 개수, 문장 길이 - 필터 리스트[n] + 1]
        
        max_pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # max_pooled_n = [배치 사이즈, 필터 개수]

        cat = self.dropout(torch.cat(max_pooled, dim = 1))
        # cat = [배치 사이즈, 필터 개수 x len(필터 리스트)]

        return self.fc(cat)  # [배치 사이즈, 1]

## 데이터 텐서 변환

In [18]:
# 문장 및 라벨 데이터 torch Tensor로 변환

processed_data = [torch.LongTensor(data).to(device) for data in processed_data]
train_label = [torch.FloatTensor([label]).to(device) for label in train_label]


# torch Tensor로 변환한 데이터 이용해 Iterator 정의

train_iter = data.DataLoader(processed_data, batch_size=params['batch_size'])
label_iter = data.DataLoader(train_label, batch_size=params['batch_size'])

## 데이터셋 이용해 CNN 모델 훈련

In [19]:
model = CNN()
model.to(device)

criterion = nn.BCEWithLogitsLoss()
criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=params['lr'])

for epoch in range(params['num_epoch']):
    model.train()
    epoch_loss = 0
    
    for (batch, label) in zip(train_iter, label_iter):
        optimizer.zero_grad()

        logits = model(batch).squeeze(1)    # [배치 사이즈]
        labels = label.view(label.size(0))  # [배치 사이즈]

        loss = criterion(logits, labels)
        epoch_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    train_loss = epoch_loss / len(train_iter)        
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')

Epoch: 01 | Train Loss: 0.466
Epoch: 02 | Train Loss: 0.345
Epoch: 03 | Train Loss: 0.304
Epoch: 04 | Train Loss: 0.274
Epoch: 05 | Train Loss: 0.243
Epoch: 06 | Train Loss: 0.211
Epoch: 07 | Train Loss: 0.180
Epoch: 08 | Train Loss: 0.161
Epoch: 09 | Train Loss: 0.146
Epoch: 10 | Train Loss: 0.137


## Captum을 이용한 모델 해석

In [20]:
# 결과 해석에 필요한 Captum API 임포트 및 정의

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

token_reference = TokenReferenceBase(reference_token_idx=pad_idx)  # 레퍼런스 생성 위한 모듈
lig = LayerIntegratedGradients(model, model.embedding)             # 결과 해석에 사용되는 IntegratedGradient 기법 모듈

In [21]:
vis_data_records_ig = []

label_vocab  = {0: '부정', 1: '긍정'}

def interpret_sentence(model, sentence, label = 0):       
    model.zero_grad()
    
    input_ids = tokenizer.encode(sentence)
    input_tokens = input_ids.tokens[:params['max_len']]
    
    input_ids = postprocess(input_ids.ids)
    input_indices_tensor = torch.LongTensor(input_ids).to(device).unsqueeze(0)

    # 단일 문장에 대한 예측 작업 수행
    pred = torch.sigmoid(model(input_indices_tensor)).item()
    pred_ind = round(pred)

    # 베이스 라인 역할을 할 Reference 생성: 주로 패딩 토큰으로 채워줌
    reference_indices = token_reference.generate_reference(params['max_len'], device=device).unsqueeze(0)

    # LayerIntegratedGradients 모듈 활용해 개별 단어의 속성값 및 델타값 근사치 계산
    attributions_ig, delta = lig.attribute(input_indices_tensor, reference_indices, n_steps=500, return_convergence_delta=True)

    print('pred: ', label_vocab[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, input_tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    

def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # 시각화 위해 샘플을 리스트에 추가
    vis_data_records.append(visualization.
                                VisualizationDataRecord(
                                    attributions,
                                    pred,
                                    label_vocab[pred_ind],
                                    label_vocab[label],
                                    label_vocab[1],
                                    attributions.sum(),       
                                    text,
                                delta
                                )
                           )

In [22]:
# 예제 문장 추가 및 분석 수행

interpret_sentence(model, '보면후회하지않을 영화였다 아쉬운부분이 없었고 마지막까지 긴장감있었다', label=1)
interpret_sentence(model, '평이 워낙 좋아 갔는데 거품이 심한듯.. 개연성도 엉망에 르부아 블랑의 뜬금없는 직관에만 의존한 스토리 전개 ;; 평점 거품이 심한듯', label=0)
interpret_sentence(model, '너무 재미있게 봤습니다. 클래식한 추리소설 느낌.타 인기영화때문에 개봉관이 적어 아쉽네요', label=1)
interpret_sentence(model, '아, 2시간 넘게 지루하고 어이없는 추격전과 의욕이 넘쳐 과욕으로 변한 연기, 게다가 예상보다 너무 뻔한 결말로 고문당함.', label=0)

pred:  긍정 ( 1.00 ) , delta:  tensor([1.2773], device='cuda:0')
pred:  부정 ( 0.00 ) , delta:  tensor([1.0413], device='cuda:0')
pred:  긍정 ( 1.00 ) , delta:  tensor([4.3315], device='cuda:0')
pred:  부정 ( 0.00 ) , delta:  tensor([2.3641], device='cuda:0')


In [23]:
# 시각화 결과 표 변환

visualization.visualize_text(vis_data_records_ig)

Target Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
긍정,긍정 (1.00),긍정,1.31,보면 후회하지 않을 영화였다 아쉬운 부분이 없었고 마지막까지 긴장감 있었다 #PAD #PAD #PAD #PAD #PAD #PAD #PAD #PAD #PAD #PAD
부정,부정 (0.00),긍정,-1.56,평이 워낙 좋아 갔는데 거품이 심 한듯 .. 개연성도 엉망 에 르 부 아 블 랑 의 뜬금없는 직 관
긍정,긍정 (1.00),긍정,1.62,너무 재미있게 봤습니다 . 클래식 한 추리 소설 느낌 . 타 인기 영화 때문에 개봉 관이 적어 아쉽네요 #PAD #PAD
부정,부정 (0.00),긍정,-1.40,"아 , 2시간 넘게 지루하고 어이없는 추격 전과 의 욕이 넘쳐 과 욕 으로 변한 연기 , 게다가 예상 보다"


### 읽을 거리

- [**Captum**](https://github.com/pytorch/captum) 라이브러리에서 사용한 **_IntegratedGradients_** 기법을 더 자세히 이해하기 위해서는 [**논문**](https://arxiv.org/abs/1703.01365)을 참조해주세요.
- 감정 분류와 관련한 더 많은 학습을 원하시는 분들은 Ben Trevett이 작성한 [**pytorch-sentiment-analysis**](https://github.com/bentrevett/pytorch-sentiment-analysis) 튜토리얼을 참조해주세요. (강추)